In [39]:
from flipside import Flipside
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import pmdarima as pm

api_key = '07473093-5657-4739-b1bb-98aeeb2f26c1'
flipside = Flipside(api_key, "https://api-v2.flipsidecrypto.xyz")


#### Query 1: Historic Price Data

In [44]:
# Calculate date range
end_date = datetime.now().date()
start_date = end_date - timedelta(days=50000)  # 2 years ago

# SQL query
sql = f"""
SELECT
    date_trunc('day', hour) as date,
    MIN(price) as low,
    MAX(price) as high,
    AVG(CASE WHEN EXTRACT(HOUR FROM hour) = 0 THEN price END) as open,
    AVG(CASE WHEN EXTRACT(HOUR FROM hour) = 23 THEN price END) as close
FROM BITCOIN.price.ez_prices_hourly
WHERE symbol = 'BTC'
    AND EXTRACT(YEAR FROM hour) BETWEEN 2020 AND 2024
GROUP BY date_trunc('day', hour)
ORDER BY date_trunc('day', hour);
"""

query_result_set = flipside.query(sql)

#### Query 2: Volume

In [43]:
volume_sql = """
SELECT
    DATE_TRUNC('day', BLOCK_TIMESTAMP) AS date,
    SUM(OUTPUT_VALUE) AS bitcoin_volume
FROM
    bitcoin.core.fact_transactions -- Replace with your actual table name
WHERE 1=1
    -- COINBASE = 'Bitcoin' -- Assuming COINBASE column indicates Bitcoin transactions
GROUP BY
    DATE_TRUNC('day', BLOCK_TIMESTAMP)
ORDER BY
    DATE_TRUNC('day', BLOCK_TIMESTAMP);
"""

Volume = flipside.query(volume_sql)
Volume = auto_paginate_result(query_result_set=Volume)
Volume = pd.DataFrame(Volume)
Volume

,date,bitcoin_volume,__row_index
0,2009-01-03T00:00:00.000Z,50.000000,0
1,2009-01-09T00:00:00.000Z,700.000000,1
2,2009-01-10T00:00:00.000Z,3050.000000,2
3,2009-01-11T00:00:00.000Z,4650.000000,3
4,2009-01-12T00:00:00.000Z,4879.000000,4
...,...,...,...
5651,2024-06-29T00:00:00.000Z,453278.115420,5651
5652,2024-06-30T00:00:00.000Z,373215.581249,5652
5653,2024-07-01T00:00:00.000Z,664824.909421,5653
5654,2024-07-02T00:00:00.000Z,634181.730253,5654


#### Function to convert into DF format

In [45]:
def auto_paginate_result(query_result_set, page_size=10000):
    """
    This function auto-paginates a query result to get all the data. It assumes 10,000 rows per page.
    In case of an error, reduce the page size. Uses numpy.
    """
    num_rows = query_result_set.page.totalRows
    page_count = np.ceil(num_rows / page_size).astype(int)
    all_rows = []
    current_page = 1
    while current_page <= page_count:
        results = flipside.get_query_results(
            query_result_set.query_id,
            page_number=current_page,
            page_size=page_size
        )

        if results.records:
            all_rows.extend(results.records)  # Use extend() to add list elements

        current_page += 1  # Increment the current page number

    return all_rows  # Return all_rows in JSON format

results = auto_paginate_result(query_result_set=query_result_set)
results = pd.DataFrame(results)
results

,date,low,high,open,close,__row_index
0,2020-01-01T00:00:00.000Z,7176.662158,7256.235116,7200.685899,7223.949583,0
1,2020-01-02T00:00:00.000Z,6961.119459,7222.235506,7207.135177,6986.753868,1
2,2020-01-03T00:00:00.000Z,6891.250830,7364.726154,6974.922052,7300.537592,2
3,2020-01-04T00:00:00.000Z,7301.334593,7375.768309,7303.722429,7362.912925,3
4,2020-01-05T00:00:00.000Z,7347.325787,7483.968613,7371.236016,7376.189807,4
...,...,...,...,...,...,...
1641,2024-06-29T00:00:00.000Z,60238.000000,61026.000000,60238.000000,60956.000000,1641
1642,2024-06-30T00:00:00.000Z,60645.000000,61984.000000,60910.000000,61820.000000,1642
1643,2024-07-01T00:00:00.000Z,62547.000000,63688.000000,62695.000000,62905.000000,1643
1644,2024-07-02T00:00:00.000Z,61852.000000,63067.000000,62902.000000,61958.000000,1644


In [59]:
results.date = pd.to_datetime(results.date)
Volume.date = pd.to_datetime(Volume.date)

btc_data = pd.merge(results, Volume, on='date', how='left')
btc_data

# Create a figure
fig = go.Figure()

# Add the first line trace
fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.close, mode='lines', name='Close Price'))

# Add the second line trace with secondary y-axis
fig.add_trace(go.Scatter(x=btc_data.date, y=btc_data.bitcoin_volume, mode='lines', name='Bitcoin Volume', yaxis='y2'))

# Customize the layout
fig.update_layout(
    title='Bitcoin Price and Volume',
    xaxis_title='Date',
    yaxis=dict(
        title='Close Price',
        titlefont=dict(color='#1f77b4'),
        tickfont=dict(color='#1f77b4')
    ),
    yaxis2=dict(
        title='Bitcoin Volume',
        titlefont=dict(color='#ff7f0e'),
        tickfont=dict(color='#ff7f0e'),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='black'
        ),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2
    ),
    width = 2100,
    height = 1000

)

# Show the plot
fig.show()

In [50]:
btc_data

,date,low,high,open,close,__row_index_x,bitcoin_volume,__row_index_y
0,2020-01-01 00:00:00+00:00,7176.662158,7256.235116,7200.685899,7223.949583,0,4.455954e+05,4010
1,2020-01-02 00:00:00+00:00,6961.119459,7222.235506,7207.135177,6986.753868,1,6.954233e+05,4011
2,2020-01-03 00:00:00+00:00,6891.250830,7364.726154,6974.922052,7300.537592,2,1.018829e+06,4012
3,2020-01-04 00:00:00+00:00,7301.334593,7375.768309,7303.722429,7362.912925,3,6.596200e+05,4013
4,2020-01-05 00:00:00+00:00,7347.325787,7483.968613,7371.236016,7376.189807,4,5.652965e+05,4014
...,...,...,...,...,...,...,...,...
1641,2024-06-29 00:00:00+00:00,60238.000000,61026.000000,60238.000000,60956.000000,1641,4.532781e+05,5651
1642,2024-06-30 00:00:00+00:00,60645.000000,61984.000000,60910.000000,61820.000000,1642,3.732156e+05,5652
1643,2024-07-01 00:00:00+00:00,62547.000000,63688.000000,62695.000000,62905.000000,1643,6.648249e+05,5653
1644,2024-07-02 00:00:00+00:00,61852.000000,63067.000000,62902.000000,61958.000000,1644,6.341817e+05,5654
